# Laboratorio 1

In [6]:
import pandas as pd
import numpy as np
import  numpy as np
import  re
import  matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
from pandas_profiling import ProfileReport

import sklearn
from sklearn import metrics, model_selection, tree

In [7]:
## Caracteristicas derivadas

def H_entropy (x):
    # Entropia de Shannon
    prob = [ float(x.count(c)) / len(x) for c in dict.fromkeys(list(x)) ] 
    H = - sum([ p * np.log2(p) for p in prob ]) 
    return H

def posicionPrimerDigito( s ):
    for i, c in enumerate(s):
        if c.isdigit():
            return i + 1
    return 0

def proporcionVocalesConsonantes (x):
    x = x.lower()
    patron_vocales = re.compile('([aeiou])')
    patron_consonantes = re.compile('([b-df-hj-np-tv-z])')
    vocales = re.findall(patron_vocales, x)
    consonantes = re.findall(patron_consonantes, x)
    try:
        proporcion = len(vocales) / len(consonantes)
    except: # Division por cero
        proporcion = 0  
    return proporcion

### Exploración de datos

In [9]:
df = pd.read_csv("dataset_pishing.csv")
df.head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 67 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   url                         11430 non-null  object 
 1   ip                          11430 non-null  int64  
 2   nb_www                      11430 non-null  int64  
 3   nb_com                      11430 non-null  int64  
 4   nb_dslash                   11430 non-null  int64  
 5   http_in_path                11430 non-null  int64  
 6   punycode                    11430 non-null  int64  
 7   port                        11430 non-null  int64  
 8   tld_in_path                 11430 non-null  int64  
 9   tld_in_subdomain            11430 non-null  int64  
 10  abnormal_subdomain          11430 non-null  int64  
 11  nb_subdomains               11430 non-null  int64  
 12  prefix_suffix               11430 non-null  int64  
 13  random_domain               114

**Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como 
“pishing”. ¿Está balanceado el dataset?**

_Esta balanceado_

In [21]:
df['status'].value_counts()     

legitimate    5715
phishing      5715
Name: status, dtype: int64

### Derivación de las características

In [22]:
# 1. Longitud
# 2. Cantidad de digitos
# 3. Calculo de entropia (Shannon), es decir la cantidad de "informacion" que se puede obtener de una variable
# 4. Posicion del primer digito en la cadena
# 5. Proporcion de vocales - consonantes

df['longitud'] = df['url'].str.len()
df['digitos'] = df['url'].str.count('[0-9]')
df['entropia'] = df['url'].apply(H_entropy)
df['proporcionVocalesConsonantes'] = df['url'].apply(proporcionVocalesConsonantes)
df['posicionPrimerDigito'] = df['url'].apply(posicionPrimerDigito)

0        0.363636
1        0.827586
2        0.517241
3        0.300000
4        0.363636
           ...   
11425    0.357143
11426    0.488889
11427    0.520833
11428    0.391304
11429    0.531818
Name: proporcionVocalesConsonantes, Length: 11430, dtype: float64